# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [3]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [4]:
prompt = f"""
Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match.
Find out who the 2023 winner was for the mens and womens singles.
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: The 2023 Wimbledon men's singles champion was Novak Djokovic, who defeated Rafael Nadal in the final. On the women's side, the champion was Ashleigh Barty, who defeated Aryna Sabalenka in the final.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [5]:
manual_grounding = """
Wimbledon 2023 Highlights
Men’s Singles: Carlos Alcaraz defeated Novak Djokovic in a thrilling five-set match (1-6, 7-6, 6-1, 3-6, 6-4), becoming the third-youngest men’s Wimbledon champion in history after Boris Becker and Björn Borg. This victory marked Alcaraz's second Grand Slam title, following his 2022 US Open win. Djokovic, who holds 23 Grand Slam titles, experienced his first loss on Centre Court in ten years.

Women’s Singles: Markéta Vondroušová won her first Grand Slam by defeating Ons Jabeur 6-4, 6-4. Vondroušová made history as the first unseeded woman to claim the Wimbledon title. Jabeur, appearing in her second consecutive Wimbledon final, expressed deep emotion after the loss.

Junior Success: Henry Searle, a 17-year-old from Wolverhampton, became the first British junior to win the boys’ title since 1962, defeating Yaroslav Demin (6-4, 6-4).

Wheelchair Tennis: Alfie Hewett and Gordon Reid claimed their fifth Wimbledon wheelchair doubles title with a comeback win (3-6, 6-0, 6-3). However, Hewett fell short in the singles final, losing to Tokito Oda (6-4, 6-2).

Other Titles:

Men’s Doubles: Neal Skupski (UK) and Wesley Koolhof defeated Marcel Granollers and Horacio Zeballos (6-4, 6-4).
Women’s Doubles: Hsieh Su-wei and Barbora Strýcová emerged victorious.
Mixed Doubles: Mate Pavić and Lyudmyla Kichenok claimed the title.
"""
prompt = f"""
Scenario: You are tasked with writing a report on the Wimbledon tennis tournament.
- Provide the names of the winners, the players they defeated, and the final match scores.
- Include a brief explanation of why the matches were significant in tennis history.

Respond concisely, and organize the information clearly.
Context: {manual_grounding}
"""


model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: Wimbledon 2023 Winners:

Men’s Singles:
- Winner: Carlos Alcaraz
- Defeated: Novak Djokovic
- Final Match Score: 1-6, 7-6, 6-1, 3-6, 6-4

Significance: Alcaraz became the third-youngest men’s Wimbledon champion in history, marking his second Grand Slam title and ending Djokovic's ten-year unbeaten streak on Centre Court.

Women’s Singles:
- Winner: Markéta Vondroušová
- Defeated: Ons Jabeur
- Final Match Score: 6-4, 6-4

Significance: Vondroušová made history as the first unseeded woman to claim the Wimbledon title, while Jabeur experienced deep emotion after the loss.

Junior Success:
- Winner: Henry Searle
- Defeated: Yaroslav Demin
- Final



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
